In [1]:
import pandas as pd
from sqlalchemy import create_engine
import time

In [2]:
# Load Data
transactions_df = pd.read_json('/home/jovyan/work/data/customer_transactions.json')
products_df = pd.read_csv('/home/jovyan/work/data/product_catalog.csv')


In [3]:
transactions_df.head(10)

,transaction_id,customer_id,product_id,quantity,price,timestamp
0,b1182d23-5bf8-4676-9f12-a0e912011caa,C001,P003,6,150,2024-02-04 13:06:35
1,3b019fcb-f96d-4403-948c-93f3028d042b,C003,P001,10,100,2024-06-22 11:42:33
2,346d2b7e-00d3-41af-b53e-0c8624aeba79,C001,P001,5,100,2024-03-12 16:16:16
3,9ad7558f-eecb-425b-864f-768db080e003,C002,P002,3,200,2024-03-29 04:01:02
4,b51d70d8-582d-4972-ba08-9ca81cc5a8a0,C003,P002,1,200,2024-02-10 09:54:36
5,b1182d23-5bf8-4676-9f12-a0e912011caa,C001,P003,6,150,2024-02-04 13:06:35
6,3b019fcb-f96d-4403-948c-93f3028d042b,C003,P001,10,100,2024-06-22 11:42:33
7,346d2b7e-00d3-41af-b53e-0c8624aeba79,C001,P001,5,100,2024-03-12 16:16:16
8,9ad7558f-eecb-425b-864f-768db080e003,C002,P002,3,200,2024-03-29 04:01:02
9,b51d70d8-582d-4972-ba08-9ca81cc5a8a0,C003,P002,1,200,2024-02-10 09:54:36


In [4]:
#drop duplicated rows
transactions_df.drop_duplicates(inplace = True)

In [5]:
transactions_df

,transaction_id,customer_id,product_id,quantity,price,timestamp
0,b1182d23-5bf8-4676-9f12-a0e912011caa,C001,P003,6,150,2024-02-04 13:06:35
1,3b019fcb-f96d-4403-948c-93f3028d042b,C003,P001,10,100,2024-06-22 11:42:33
2,346d2b7e-00d3-41af-b53e-0c8624aeba79,C001,P001,5,100,2024-03-12 16:16:16
3,9ad7558f-eecb-425b-864f-768db080e003,C002,P002,3,200,2024-03-29 04:01:02
4,b51d70d8-582d-4972-ba08-9ca81cc5a8a0,C003,P002,1,200,2024-02-10 09:54:36


In [6]:
products_df.head(10)

,product_id,product_name,category,price
0,P001,Product 1,Category A,-100.0
1,P002,Product 2,Category B,200.0
2,P003,Product 3,Category A,150.0
3,P004,NaN,Category C,300.0
4,P005,Product 5,Category C,invalid_price
5,P001,Product 1,Category A,100.0


In [7]:
# remove price = -100.00 , edit name of Product 4
products_df['price'] = pd.to_numeric(products_df['price'], errors='coerce')
products_df = products_df[products_df['price'] != -100.0]
products_df.at[3, 'product_name'] = "Product 4"

In [8]:
products_df

,product_id,product_name,category,price
1,P002,Product 2,Category B,200.0
2,P003,Product 3,Category A,150.0
3,P004,Product 4,Category C,300.0
4,P005,Product 5,Category C,NaN
5,P001,Product 1,Category A,100.0


In [9]:
# Database connection
engine = create_engine('postgresql://user:password@db:5432/shopsmart')
# Create tables and insert data
transactions_df.to_sql('transactions', engine, if_exists='replace', index=False)
products_df.to_sql('products', engine, if_exists='replace', index=False)

5

# What are the top 2 best-selling products?
## ANS :  P001,P003

In [10]:
query_top2 = """
SELECT  t.product_id, p.product_name, SUM(t.quantity) as n_quantity
FROM transactions t inner join products p
on t.product_id =  p.product_id
GROUP BY t.product_id, p.product_name
ORDER BY n_quantity DESC
LIMIT 2 
"""
top_2_products = pd.read_sql(query_top2, engine)
print(top_2_products)

  product_id product_name  n_quantity
0       P001    Product 1        15.0
1       P003    Product 3         6.0


# What is the average order value per customer?
## ANS : C001 = 700 , C002 = 600 , C003 = 600

In [11]:
query_AVG = """
SELECT customer_id, AVG(quantity * price) as average_order_value
FROM transactions
GROUP BY customer_id
ORDER BY customer_id;
"""
average_order_value = pd.read_sql(query_AVG, engine)
print(average_order_value)

  customer_id  average_order_value
0        C001                700.0
1        C002                600.0
2        C003                600.0


# What is the total revenue generated per product category?
## ANS : A = 2400 , B = 800 

In [12]:
query_revenue = """
SELECT p.category, SUM(t.quantity * t.price) as total_revenue
FROM transactions t
INNER JOIN products p ON t.product_id = p.product_id
GROUP BY p.category
ORDER BY total_revenue DESC;
"""
total_revenue = pd.read_sql(query_revenue, engine)
print(total_revenue)

     category  total_revenue
0  Category A         2400.0
1  Category B          800.0
